In [1]:
import numpy as np
from dataclasses import dataclass
import math
from typing import Literal, List
import gsw 

PSI_TO_DBAR = 0.6894759
sea_level_pressure = 14.7
DBAR_TO_PSI = 1.450377

In [2]:
class PressureCoefficients:
    C1: float
    C2: float
    C3: float
    D1: float
    D2: float
    T1: float
    T2: float
    T3: float
    T4: float
    T5: float
    Slope: float
    Offset: float
    AD590M: float
    AD590B: float
    
pres_coefs = PressureCoefficients()
pres_coefs.C1 = -4.212464e+004
pres_coefs.C2 = -1.972709e-001
pres_coefs.C3 = 1.365930e-002

pres_coefs.D1 = 3.604100e-002
pres_coefs.D2 = 0.000000e+000

pres_coefs.T1 = 3.046128e+001
pres_coefs.T2 = -3.791930e-004
pres_coefs.T3 = 4.202260e-006
pres_coefs.T4 = 3.386190e-009
pres_coefs.T5 = 0.000000e+000

pres_coefs.Slope = 1.00000000
pres_coefs.Offset = -0.20000
pres_coefs.AD590M = 1.279410e-002
pres_coefs.AD590B = -9.215740e+000

hexstrings = [
    "0CD08410767B80458F0CCA6F1107AE7D58A3987C41F0DEF2FFFFBE0000BB29512A089DB7004D638E5C91C465",
    "0CD0811076768045960CCA6F1107B27D48A3986C40F0DEF2FFFFBD0000BB29512A089DB7004D638F5C91C465",
    "0CD08710767B8045960CCA6D1107B17D48A3986C40F10EF0FFFFBE0000BB29512A089DB7004D63905D91C465",
    "0CD08710767A80458F0CCA6C1107AE7D58A3986C3FF16EE7FFFFBE0000BB29512A089DB7004D63915D91C465",
    "0CD08A10767680459E0CCA6F1107B27D58A3987C40F18EE3FFFFBE0000BB29512A089DB7004D63925D91C465",
    "0CD08410767A8045960CCA6C1107B17D58A3988C41F18EE3FFFFBE0000BB29512A089DB7004D63935D91C465",
    "0CD0811076798045960CCA6A1107B17D58A498AC41F18EE1FFFFBE0000BB29512A089DB7004D63945D91C465",
    "0CD08410767C80458F0CCA6A1107B07D48A398BC40F15ED4FFFFBE0000BB29512A089DB7004D63955D91C465",
    "0CD08710767A8045960CCA6F1107B17D48A498CC41F13ECEFFFFBE0000BB29512A089DB7004D63965D91C465",
    "0CD08110767980459D0CCA6F1107B07D48A398DC41F13ECEFFFFBD0000BB29512A089DB7004D63975D91C465",
    "0CDE1C1133E58052270CD78511B3F07EA8B7F68042E9CB5FFFFFBD0000C6295127089DBA004D2387DC91C465",
    "0CDE181133BE8052360CD78B11B3DC7E98B7F68040EBEB2EFFFFBE0000C6295127089DBA004D2388DC91C465",
    "0CDE241133E280522A0CD78811B3D07EA8B7F68040ED4B0FFFFFBE0000C6295127089DBA004D3389DC91C465",
    "0CDE2111343280522F0CD78511B38C7E98B7F68040ED7B0BFFFFBE0000C5295127089DBA004D238ADC91C465",
    "0CDE2511345F80522F0CD78811B3177E98B7F68041ED7B0DFFFFBF0000C5295127089DBA004D238BDC91C465"
]
bytes_list_array = [[hex_string[i:i+2] for i in range(0, len(hex_string), 2)] for hex_string in hexstrings]

In [3]:
def pressure_from_frequency(
    pressure_counts: np.ndarray, 
    temp_counts: np.ndarray, 
    coefs: PressureCoefficients,
    units: Literal["dbar", "psia"] = "psia"
):
    """
    Berechnet den Druck [psia] aus Digiquartz-Frequenz und Temperaturcount (arrays erlaubt).

    :param freq_hz: gemessene Druckfrequenz [Hz]
    :param temp_freq_hz: gemessene Temperaturcount
    :param coefs: PressureCoefficients
    :return: Druck in psia
    """

    U = coefs.AD590M * temp_counts + coefs.AD590B
    print("U:", U)
    C = coefs.C1 + coefs.C2 * U + coefs.C3 * U **2
    D = coefs.D1 + coefs.D2 * U
    T0 = (
        coefs.T1 
        + coefs.T2 * U 
        + coefs.T3 * U **2 
        + coefs.T4 * U **3 
        + coefs.T5 * U **4
    )
    # Pressure Periode in µs
    T = 10**6 / pressure_counts
    print("T:", T)
    
    pressure = C * (1 - (T0**2/T**2)) * (1 - D * (1 -(T0**2/T**2)))
    pressure = pressure * coefs.Slope + (coefs.Offset * DBAR_TO_PSI) - sea_level_pressure
    
    if units == "dbar":
        pressure *= PSI_TO_DBAR
    
    return pressure


In [4]:
pressure_counts = []

for byte_list in bytes_list_array:
    count = int(byte_list[6], 16) * 256 + int(byte_list[7], 16) + int(byte_list[8], 16) / 256
    pressure_counts.append(count)

pressure_counts = np.array(pressure_counts)

print(pressure_counts)
temperature_counts = []

for hex_string in hexstrings:
    temperature_value = int(hex_string[74:77],16)
    temperature_counts.append(temperature_value)

temperature_counts = np.array(temperature_counts)
print(temperature_counts)

pressure = pressure_from_frequency(pressure_counts, temperature_counts, coefs=pres_coefs)


print("Druck in psi", pressure, "gewünscht: 1.667")
pressure *= PSI_TO_DBAR # Druck von psia zu db laut seabird

print("Druck in dbar:", pressure, "gewünscht: 1.149")

[32837.55859375 32837.5859375  32837.5859375  32837.55859375
 32837.6171875  32837.5859375  32837.5859375  32837.55859375
 32837.5859375  32837.61328125 32850.15234375 32850.2109375
 32850.1640625  32850.18359375 32850.18359375]
[1238 1238 1238 1238 1238 1238 1238 1238 1238 1238 1234 1234 1235 1234
 1234]
U: [6.6233558 6.6233558 6.6233558 6.6233558 6.6233558 6.6233558 6.6233558
 6.6233558 6.6233558 6.6233558 6.5721794 6.5721794 6.5849735 6.5721794
 6.5721794]
T: [30.45293386 30.4529085  30.4529085  30.45293386 30.45287952 30.4529085
 30.4529085  30.45293386 30.4529085  30.45288314 30.44125913 30.44120483
 30.44124827 30.44123017 30.44123017]
Druck in psi [ 1.66650044  1.73668555  1.73668555  1.66650044  1.81689718  1.73668555
  1.73668555  1.66650044  1.73668555  1.80687072 34.0446547  34.19511762
 34.06328618 34.12490155 34.12490155] gewünscht: 1.667
Druck in dbar: [ 1.14901189  1.19740283  1.19740283  1.14901189  1.25270682  1.19740283
  1.19740283  1.14901189  1.19740283  1.24579382